In [1]:
import transformers
import random
import tensorflow as tf
import numpy as np

In [15]:
pipe = transformers.pipeline('text-generation')

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)


In [16]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

def clean_text(txt):
    return ' '.join(txt.strip().split())

In [17]:

prompt="what is the meaning of life?"
output = pipe(prompt)
output

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'what is the meaning of life?\n\nWhat would an understanding of life become when we become aware of it?\n\nHow does life be understood when we cease to be aware of it?\n\nHow does life become understood when we cease to'}]

In [18]:
responses = list(map(lambda x: clean_text(
    x['generated_text'][len(prompt):]), output))
responses


['What would an understanding of life become when we become aware of it? How does life be understood when we cease to be aware of it? How does life become understood when we cease to']

In [19]:
def build_ranker_dict():
    """Build dictionary of ranker weights and pipelines."""

    human_vs_rand_weight = None
    human_vs_machine_weight = None
    updown_weight = None
    depth_weight = None
    width_weight = None

    ranker_dict = dict()
    if human_vs_rand_weight is not None:
        ranker_dict['human_vs_rand'] = dict(
            pipe=transformers.pipeline(
                'sentiment-analysis', model='microsoft/DialogRPT-human-vs-rand'),
            weight=human_vs_rand_weight,
            group='prior'
        )
    if human_vs_machine_weight is not None:
        ranker_dict['human_vs_machine'] = dict(
            pipe=transformers.pipeline(
                'sentiment-analysis', model='microsoft/DialogRPT-human-vs-machine'),
            weight=human_vs_machine_weight,
            group='prior'
        )
    if updown_weight is not None:
        ranker_dict['updown'] = dict(
            pipe=transformers.pipeline(
                'sentiment-analysis', model='microsoft/DialogRPT-updown'),
            weight=updown_weight,
            group='cond'
        )
    if depth_weight is not None:
        ranker_dict['depth'] = dict(
            pipe=transformers.pipeline(
                'sentiment-analysis', model='microsoft/DialogRPT-depth'),
            weight=depth_weight,
            group='cond'
        )
    if width_weight is not None:
        ranker_dict['width'] = dict(
            pipe=transformers.pipeline(
                'sentiment-analysis', model='microsoft/DialogRPT-width'),
            weight=width_weight,
            group='cond'
        )
    return ranker_dict

ranker_dict = build_ranker_dict()
ranker_dict

{}

In [20]:
def pick_best_response(prompt, responses, ranker_dict):
    if len(ranker_dict) == 0:
        return random.choice(responses)

m = pick_best_response(prompt, responses, ranker_dict)
m

'What would an understanding of life become when we become aware of it? How does life be understood when we cease to be aware of it? How does life become understood when we cease to'

In [58]:
from transformers import AutoModelWithLMHead, TFAutoModelForCausalLM, AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-large')
model = TFAutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-large')


Downloading: 100%|██████████| 2.88G/2.88G [03:27<00:00, 14.9MB/s]
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at microsoft/DialoGPT-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
